In [1]:
import pandas as pd
import numpy as np
import re
import pytz
import os
from pathlib import Path
import sys
sys.path.append("/home/jovyan/shared/service-data")

from src.clean import clean_percentage, normalize_string, standardize_column_names, clean_fiscal_yr
from src.load import load_csv_from_raw
from src.export import export_to_csv
from src.merge import merge_si, merge_ss

base_dir = Path.cwd()
parent_dir = base_dir.parent

In [2]:
si = merge_si()
ss = merge_ss()

Exported dept.csv to /home/jovyan/shared/service-data/outputs/utils
Exported si_all.csv to /home/jovyan/shared/service-data/outputs/utils
Exported sid_list.csv to /home/jovyan/shared/service-data/outputs/utils
Exported dept.csv to /home/jovyan/shared/service-data/outputs/utils
Exported ss_all.csv to /home/jovyan/shared/service-data/outputs/utils
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12752 entries, 0 to 12751
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   fiscal_yr                   12752 non-null  object 
 1   service_id                  12752 non-null  object 
 2   service_name_en             12752 non-null  object 
 3   service_name_fr             12752 non-null  object 
 4   service_standard_id         12751 non-null  object 
 5   service_standard_en         12734 non-null  object 
 6   service_standard_fr         12681 non-null  object 
 7   type                     

In [3]:
si = si.loc[si['service_scope_ext_or_ent']]

si_inscope = si.set_index(['fiscal_yr','service_id'])

ss_inscope = ss.set_index(['fiscal_yr', 'service_id']).merge(si_inscope['service_scope_ext_or_ent'], how='left', left_index=True, right_index=True)
ss_inscope = ss_inscope.loc[ss_inscope['service_scope_ext_or_ent'].notna()].reset_index()


In [15]:
ss = ss_inscope


maf2 = ss.loc[:, ['fiscal_yr', 'service_standard_id', 'department_en','department_fr', 'org_id', 'target_met']].dropna()
    
maf2_num = maf2[maf2['target_met']=='Y'].groupby(['fiscal_yr', 'department_en','department_fr', 'org_id'])['service_standard_id'].count().reset_index()
maf2_denom = maf2.groupby(['fiscal_yr', 'department_en','department_fr', 'org_id'])['service_standard_id'].count().reset_index()

maf2 = pd.merge(
    maf2_num,
    maf2_denom,
    suffixes=['_met','_total'],
    on=['fiscal_yr', 'department_en','department_fr', 'org_id'],
    how='outer'
)

maf2['maf2_score'] = (maf2['service_standard_id_met']/maf2['service_standard_id_total'])*100

maf2.loc[maf2['fiscal_yr'] == '2023-2024', ['service_standard_id_total']].sum()

service_standard_id_total    1744
dtype: int64



ifoi_en = pd.read_csv(parent_dir / "inputs" / "ifoi_en.csv")
ifoi_fr = pd.read_csv(parent_dir / "inputs" / "ifoi_fr.csv")

ifoi_en = ifoi_en.set_index(ifoi_en.columns[0], drop=True).add_suffix('_en')
ifoi_fr = ifoi_fr.set_index(ifoi_fr.columns[0], drop=True).add_suffix('_fr')

ifoi_en.set_index(ifoi_en.columns[0], drop=True)
ifoi_fr.set_index(ifoi_fr.columns[0], drop=True)

ifoi = pd.concat([ifoi_en, ifoi_fr], axis=1)

# Extract column lists
en_cols = ifoi_en.columns.tolist()
fr_cols = ifoi_fr.columns.tolist()

# Interleave them by index
merged_cols = [col for pair in zip(en_cols, fr_cols) for col in pair]

# Apply new column order
ifoi = standardize_column_names(ifoi[merged_cols].reset_index())

ifoi